In [9]:
%load_ext autoreload
%autoreload 2

import os as os
import glob as glob
from utils import load_mol2, load_pdb, map_atoms, map_residues
from process import convert_parameters

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
base_path = 'a-bam-p/'
directories = []
for window in range(0, 14):
    directories.append('a' + f'{window:02d}')
for window in range(0, 1):
    directories.append('r' + f'{window:02d}')
for window in range(0, 46):
    directories.append('u' + f'{window:02d}')

In [13]:
reference_mol = load_mol2(base_path + 'a00/' + 'generated/' + 'reference.mol2')
target_mol = load_mol2(base_path + 'a00/' + 'generated/' + 'target.mol2')
atom_mapping = map_atoms(reference_mol, target_mol)

reference_mol = load_pdb(base_path + 'a00/' + 'generated/' + 'reference.pdb')
target_mol = load_pdb(base_path + 'a00/' + 'generated/' + 'target.pdb')
residue_mapping = map_residues(atom_mapping, reference_mol, target_mol)

Determining mapping...


In [ ]:
for directory in directories:
    if not os.path.exists(base_path + directory + '/generated/'):
        os.makedirs(base_path + directory + '/generated/')
    print(f'\n{directory}')
    convert_parameters(source_directory=base_path + directory + '/original/',
                       source_crd='full.crds',
                       source_top='full.hmr.topo',
                       destination_directory=base_path + directory + '/generated/',
                       destination_crd='smirnoff.inpcrd',
                       destination_top='smirnoff.prmtop',
                       host_resname='MGO',
                       guest_resname='BAM',
                       atom_mapping=atom_mapping,
                       residue_mapping=residue_mapping)


a00
PDB file written by cpptraj.
First water residue = 160
Found first water CONECT entry at line = 8322
MOL2 file written by cpptraj.
MOL2 file written by antechamber.
126 atoms in structure, 126 unique atom names.
17 atoms in structure, 17 unique atom names.
Structure looks good.
Check if the host-guest parameter file already exists...
Check if the host-guest coordinate file already exists...
Water and ion PDB file written by cpptraj.
Writing dummy atom `frcmod`.
Writing dummy atom `mol2`.
Water and ion parameters and coordinates written by tleap.
Check if solvated parameter file already exists...
Check if solvated coordinate file already exists...
Check if file exists...
Check if file exists...
["':1-3", '|', ':10@C4', '|', ":10@N1',"] → ':8-10 | :7@C4 | :7@N1',
["':1-3", '|', ':10@C4', '|', ":10@N1',"] → ':8-10 | :7@C4 | :7@N1',
["':1-3", '|', ':10@C4', '|', ":10@N1',"] → ':8-10 | :7@C4 | :7@N1',
["':1-3',"] → ':8-10',
["':1-3',"] → ':8-10',
1,13,              → 144,10            

In [ ]:
with open('generated/a-bam-p/md.sh', 'w') as file:
    file.write('#!/usr/bin/env bash\n')
    file.write('source $AMBERHOME/amber.sh\n')
    file.write('pmemd.cuda -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c smirnoff.inpcrd -i mini.in '
               '-o mini.out -r mini.rst -inf /dev/null\n')
    file.write('pmemd.cuda -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c mini.rst -i therm1.in '
               '-o therm1.out -r therm1.rst -inf therm1.mdinfo\n')
    file.write('pmemd.cuda -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c therm1.rst -i therm2.in '
               '-o therm2.out -r therm2.rst -inf therm2.mdinfo\n')
    file.write('cp therm2.rst eqnpt.rst\n')
    file.write('\n')
    for iteration in range(1, 101):
        file.write(f'pmemd.cuda -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c eqnpt.rst -i eqnpt.in '
                   f'-o eqnpt.{iteration:02d}.out -r eqnpt.{iteration:02d}.rst -inf /dev/null >& eqnpt.log \n')
        file.write(f'cat eqnpt.{iteration:02d}.out >> eqnpt.all.out\n')
        file.write(f'cp eqnpt.{iteration:02d}.rst eqnpt.rst\n')
        file.write('\n')
        

    file.write('cp eqnpt.100.rst md.000.rst\n')
    file.write('\n')
    for iteration in range(1, 201):
        file.write(f'pmemd.cuda -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c md.{(iteration-1):03d}.rst -i mdin '
                   f'-o mdout.{iteration:03d} -r md.{iteration:03d}.rst -x traj.{iteration:03d}.nc -inf mdinfo.{iteration:03d} -e mden.{iteration:03d} \n')
        file.write('\n')

In [ ]:
with open('asdfasd') as file:
    
    string = '''
    #!/bin/bash
    #PBS -l walltime=2:00:00,nodes=1:ppn=2 -q home-gibbs
    #PBS -j oe -r n -m a -M slochower@gmail.com
    #PBS -N 

    module swap intel gnu
    module load cuda
    module load python
    module load scipy/2.7

    PATH="/home/henrikse/gnu44-bin:/opt/mvapich2/gnu/ib/bin:$PATH"
    CUDA_HOME="/opt/cuda/7.5.18"
    LD_LIBRARY_PATH="${CUDA_HOME}/lib64:/opt/mvapich2/gnu/ib/lib:$LD_LIBRARY_PATH"
    MV2_ENABLE_AFFINITY=0
    SCRDIR=/oasis/tscc/scratch/davids4/hiv-protease/
    source /home/henrikse/amber16gnu/tsccamber.sh

    # Synchronize all the files in the current directory with the scratch directory
    mkdir -p $SCRDIR
    rsync -avL $PBS_O_WORKDIR $SCRDIR

    # Run the super duper python script
    cd $SCRDIR/1hhp_apo
    bash md.sh
    ''' 
